# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [1]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [2]:
dataset = pd.read_csv("./data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [3]:
X.head()

,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,4700,88026.0,115506.0,6.0,0.0,0.0,182.248332,0.0,27.0,29.209023
1,19300,39926.0,101208.0,4.0,0.0,0.0,140.051638,0.0,14.0,31.545694
2,5700,71556.0,79538.0,2.0,0.0,0.0,92.643085,0.0,15.0,41.210012
3,13000,44875.0,57713.0,0.0,1.0,0.0,184.990324,1.0,12.0,28.602076
4,19300,72752.0,106084.0,11.0,0.0,0.0,193.707100,1.0,13.0,30.686106



Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

1. El conjunto de datos Home Equity (HMEQ) contiene la línea de base y la información de rendimiento del préstamo  para 5,960 préstamos con garantía hipotecaria.
2. TARGET (MALO) es una variable binaria que indica si un solicitante finalmente incumplió o delinquió gravemente.
3. LOAN Monto de la solicitud de préstamo MORTDUE Monto adeudado por hipoteca existente VALOR Valor de la propiedad actual YOJ Años en el trabajo actual DEROG Número de informes despectivos importantes DELINQ Número de líneas de crédito morosas CLAGE Edad de la línea comercial más antigua en meses NINQ Número de líneas de crédito recientes CLNO Número de líneas de crédito DEBTINC Relación deuda / ingresos
4. A simple vista las var

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
LOAN,1483.0,19019.487525,10755.943240,1700.000000,12050.000000,17000.000000,23700.000000,89800.000000
MORTDUE,1483.0,76396.345920,45995.563918,5627.000000,48717.500000,67389.000000,94648.500000,399412.000000
VALUE,1483.0,107329.562374,55261.111710,21144.000000,71156.000000,94504.000000,124155.500000,512650.000000
YOJ,1483.0,8.948753,7.577317,0.000000,3.000000,7.000000,13.000000,41.000000
DEROG,1483.0,0.186109,0.696353,0.000000,0.000000,0.000000,0.000000,10.000000
DELINQ,1483.0,0.326365,0.925001,0.000000,0.000000,0.000000,0.000000,10.000000
CLAGE,1483.0,179.558267,85.433422,0.486711,116.439339,174.506408,233.763633,1168.233561
NINQ,1483.0,1.140931,1.683761,0.000000,0.000000,1.000000,2.000000,11.000000
CLNO,1483.0,21.859069,9.452400,0.000000,16.000000,21.000000,27.000000,65.000000
DEBTINC,1483.0,34.538331,9.566838,0.838118,29.293916,35.302774,39.200625,144.189001


Podemos ver que todas las features son positivas, por lo tanto la media de la norma l2 es la media de las features

In [6]:
#preprocesing, normalizamos las features por la media de la norma l2
X_train_n = pd.DataFrame()
X_test_n = pd.DataFrame()
features = X_train.columns
for f in features:
    mean_train = X_train[f].mean()
    X_train_n[f] = X_train[f]/mean_train
    mean_test = X_test[f].mean()
    X_test_n[f] = X_test[f]/mean_test

In [7]:
#Incorporamos la standarización de las features
model = make_pipeline(StandardScaler(), SGDClassifier(random_state = 0)) 
#model = SGDClassifier(random_state = 0)

model.fit(X_train_n, y_train)

y_train_pred = model.predict(X_train_n)

print('Resultados para el conjunto de entrenamiento:\n')
print(classification_report(y_train, y_train_pred))
print('Matriz de confusión:')
confusion_matrix(y_train, y_train_pred)

Resultados para el conjunto de entrenamiento:

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1232
           1       0.71      0.42      0.53       251

    accuracy                           0.87      1483
   macro avg       0.80      0.69      0.73      1483
weighted avg       0.86      0.87      0.86      1483

Matriz de confusión:


array([[1190,   42],
       [ 146,  105]])

In [8]:
y_test_pred = model.predict(X_test_n)
print('Resultados para el conjunto de test:\n')
print(classification_report(y_test, y_test_pred))
print('Matriz de confusión:')
confusion_matrix(y_test, y_test_pred)

Resultados para el conjunto de test:

              precision    recall  f1-score   support

           0       0.89      0.96      0.92       313
           1       0.62      0.34      0.44        58

    accuracy                           0.87       371
   macro avg       0.76      0.65      0.68       371
weighted avg       0.85      0.87      0.85       371

Matriz de confusión:


array([[301,  12],
       [ 38,  20]])

Podemos ver que sin ajustar los hiperparámetros del SGDClassifier tiene un desempeño bajo en la variable objetivo 1 tanto en el set de entranamiento como en el de test, este desempeño no se modifica si tomo los valores de las features sin normalizar pero recall decae considerablemete para TARGET 1 si no estandarizo las features

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterSampler
import numpy as np

In [10]:
param_grid = {
    'loss': [
        'hinge',        # SVM
        'log',          # logistic regression
        #'preceptron',  # perceptron (not supported)
        'modified_huber',
        'squared_hinge',
        'squared_loss',
        'huber'
    ],
    'alpha': 10.0**-np.arange(1,7),
    'learning_rate': [
        'constant',
        'optimal',
        'invscaling',
        'adaptive'
    ],
    'eta0': [
        1e-2,
        1e-1,
        1,
        1e1,
        1e2,
        1e3,
        1e4
    ]
}

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
model = SGDClassifier(random_state = 0, max_iter = 2000)

cv = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)

#cv = make_pipeline(StandardScaler(), GridSearchCV(model, param_grid, scoring='accuracy', cv=5))

cv.fit(X_train_n, y_train);    

In [13]:
#results = cv['gridsearchcv'].cv_results_
results = cv.cv_results_
df = pd.DataFrame(results)
df[['param_loss', 'param_alpha','param_learning_rate','param_eta0', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_loss,param_alpha,param_learning_rate,param_eta0,mean_test_score,std_test_score,rank_test_score
0,hinge,0.1,constant,0.01,0.837510,0.017589,422
1,log,0.1,constant,0.01,0.844920,0.010510,331
2,modified_huber,0.1,constant,0.01,0.799741,0.027023,692
3,squared_hinge,0.1,constant,0.01,0.779416,0.084530,758
4,squared_loss,0.1,constant,0.01,0.770737,0.039116,783
...,...,...,...,...,...,...,...
1003,log,1e-06,adaptive,10000,0.877280,0.014549,8
1004,modified_huber,1e-06,adaptive,10000,0.859098,0.037079,116
1005,squared_hinge,1e-06,adaptive,10000,0.763309,0.051937,814
1006,squared_loss,1e-06,adaptive,10000,0.860422,0.006861,98


In [14]:
df[df['rank_test_score']<2][['param_loss', 'param_alpha','param_eta0','param_learning_rate', 
                             'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_loss,param_alpha,param_eta0,param_learning_rate,mean_test_score,std_test_score,rank_test_score
978,hinge,1e-06,1000,adaptive,0.880651,0.018672,1


In [15]:
#best_model = cv['gridsearchcv'].best_estimator_
best_model = cv.best_estimator_

y_train_pred = best_model.predict(X_train_n)

print('Resultados para el conjunto de entrenamiento:\n')
print(classification_report(y_train, y_train_pred))
print('Matriz de confusión:')
confusion_matrix(y_train, y_train_pred)

Resultados para el conjunto de entrenamiento:

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      1232
           1       0.82      0.36      0.50       251

    accuracy                           0.88      1483
   macro avg       0.85      0.67      0.71      1483
weighted avg       0.87      0.88      0.86      1483

Matriz de confusión:


array([[1212,   20],
       [ 161,   90]])

In [16]:
y_test_pred = best_model.predict(X_test_n)

print('Resultados para el conjunto de test:\n')
print(classification_report(y_test, y_test_pred))
print('Matriz de confusión:')
confusion_matrix(y_test, y_test_pred)

Resultados para el conjunto de test:

              precision    recall  f1-score   support

           0       0.89      0.99      0.93       313
           1       0.82      0.31      0.45        58

    accuracy                           0.88       371
   macro avg       0.85      0.65      0.69       371
weighted avg       0.87      0.88      0.86       371

Matriz de confusión:


array([[309,   4],
       [ 40,  18]])

Con una búsqueda de parámetros, mediante GridSearch, pudimos mejorar el modelo para predecir TARGET 1, esto se observa sobre todo en el set de test. El valor de accuracy mejora en 0.01 con respecto entrenar el modelo SGDClassifier con los parámetros por defualt. Al implementar StandardScaler vimos una caída del accuracy de 0.08 puntos

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [17]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [34]:
y_train_pred = clf.predict(X_train)

print('Resultados para el conjunto de entrenamiento:\n')
print(classification_report(y_train, y_train_pred))
print('Matriz de confusión:')
confusion_matrix(y_train, y_train_pred)

Resultados para el conjunto de entrenamiento:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1232
           1       1.00      1.00      1.00       251

    accuracy                           1.00      1483
   macro avg       1.00      1.00      1.00      1483
weighted avg       1.00      1.00      1.00      1483

Matriz de confusión:


array([[1232,    0],
       [   0,  251]])

In [32]:
y_test_pred = clf.predict(X_test)

print('Resultados para el conjunto de test:\n')
print(classification_report(y_test, y_test_pred))
print('Matriz de confusión:')
confusion_matrix(y_test, y_test_pred)

Resultados para el conjunto de test:

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       313
           1       0.62      0.64      0.63        58

    accuracy                           0.88       371
   macro avg       0.77      0.78      0.78       371
weighted avg       0.88      0.88      0.88       371

Matriz de confusión:


array([[290,  23],
       [ 21,  37]])

Podemos ver que DecisionTreeClassifier ajusta con los parámetros por default de manera similar que el SGDClassifier aun después de hacer una búsqueda de parámetros para el conjunto de test y predice de manera perfecta para el conjunto de entrenamiento

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [22]:
param_grid2 = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [1, 2,3, 4, 5, 6, 7, 8, 9],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8]
}

In [23]:
model = DecisionTreeClassifier(random_state = 0)
cv = GridSearchCV(model, param_grid2, scoring='accuracy', cv=5)
cv.fit(X_train, y_train); 

In [24]:
results = cv.cv_results_
df = pd.DataFrame(results)
df[['param_criterion', 'param_splitter','param_max_depth','param_min_samples_leaf', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_criterion,param_splitter,param_max_depth,param_min_samples_leaf,mean_test_score,std_test_score,rank_test_score
0,gini,best,1,1,0.869194,0.009254,107
1,gini,random,1,1,0.832089,0.012051,273
2,gini,best,1,2,0.869194,0.009254,107
3,gini,random,1,2,0.832089,0.012051,273
4,gini,best,1,3,0.869194,0.009254,107
...,...,...,...,...,...,...,...
283,entropy,random,9,6,0.853010,0.015979,231
284,entropy,best,9,7,0.842224,0.020619,272
285,entropy,random,9,7,0.858392,0.015421,212
286,entropy,best,9,8,0.847604,0.016294,266


In [25]:
df[df['rank_test_score']<2][['param_criterion', 'param_splitter','param_max_depth','param_min_samples_leaf', 
                             'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_criterion,param_splitter,param_max_depth,param_min_samples_leaf,mean_test_score,std_test_score,rank_test_score
260,entropy,best,8,3,0.881991,0.011551,1


In [31]:
best_model = cv.best_estimator_

y_train_pred = best_model.predict(X_train)

print('Resultados para el conjunto de entrenamiento:\n')
print(classification_report(y_train, y_train_pred))
print('Matriz de confusión:')
confusion_matrix(y_train, y_train_pred)

Resultados para el conjunto de entrenamiento:

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1232
           1       0.91      0.66      0.76       251

    accuracy                           0.93      1483
   macro avg       0.92      0.82      0.86      1483
weighted avg       0.93      0.93      0.93      1483

Matriz de confusión:


array([[1215,   17],
       [  86,  165]])

In [33]:
y_test_pred = best_model.predict(X_test)
print('Resultados para el conjunto de test:\n')
print(classification_report(y_test, y_test_pred))
print('Matriz de confusión:')
confusion_matrix(y_test, y_test_pred)

Resultados para el conjunto de test:

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       313
           1       0.66      0.57      0.61        58

    accuracy                           0.89       371
   macro avg       0.79      0.76      0.77       371
weighted avg       0.88      0.89      0.88       371

Matriz de confusión:


array([[296,  17],
       [ 25,  33]])

Cuando utilizamos el GridSearch para el árbol de decisión podemos ver que los valores de accuracy del modelo decaen para el conjunto de entrenamiento en 0.07 y aumentan para el conjunto de test en 0.01 ya que mejoran para TARGET 0 levemente y decaen para TARGET 1 también levemente

### Ejercicio 3.3: Inspección del Modelo

De los dos modelos implementados (SGDClassifier y DecisionTreeClassifier) podemos ver que el que mejor ajusta el set de datos es el DecisionTreeClassifier cuando utilizamos los valores por default